# Frame Analysis

In [47]:
import pandas as pd
import os
import os.path as path
import json
from pprint import pprint
from collections import Counter
import ast
import numpy as np
from pathlib import Path



def read_json(path):
    import json
    with open(path) as json_file:
        data = json.load(json_file)
    return data


root = path.abspath(path.join(os.getcwd(),"../.."))

In [48]:
summary = read_json(root+'/data/processed/topic_frame_summary.json')
print(len(summary))
summary

440


[{'topic': '$700 billion US economic bailout',
  'topic_id': 0,
  'frame_info': {'alternatives': {'stance': {'Con': 1, 'Pro': 1}},
   'economics': {'stance': {'Con': 5, 'Pro': 6}},
   'immediacy': {'stance': {'Con': 2, 'Pro': 2}},
   'intervention': {'stance': {'Con': 3, 'Pro': 3}},
   'moral hazard': {'stance': {'Con': 1, 'Pro': 1}},
   'powers': {'stance': {'Con': 2, 'Pro': 2}},
   'public opinion': {'stance': {'Con': 2, 'Pro': 3}},
   'taxpayers': {'stance': {'Con': 4, 'Pro': 2}},
   'wallstreet': {'stance': {'Con': 3, 'Pro': 3}},
   'total_pro': 23,
   'total_con': 23,
   'num_of_stances': 46,
   'frames_id': [11, 4, 6, 2, 6, 6, 5, 4, 2]}},
 {'topic': '2009 US economic stimulus',
  'topic_id': 8,
  'frame_info': {'bi-partisanship': {'stance': {'Pro': 2}},
   'buy american': {'stance': {'Con': 1}},
   'consequences': {'stance': {'Con': 3, 'Pro': 2}},
   'consumer confidence': {'stance': {'Con': 1, 'Pro': 1}},
   'debt': {'stance': {'Con': 2, 'Pro': 2}},
   'good/bad stimulus': {'sta

##  "How many topics we can use for our exp, where we can control on frames (i.e., we have enough pro and con args for a single frame)?"

For a given a topic, what frames can we select that their respective stances (Pro/Con) have at least eight instances each one? For example:

For topic:

    · 700 mile US Mexico border fence
         · 'border security': {'stance': {'Con': 9, 'Pro': 9}} ✅ 
         · 'economics': {'stance': {'Con': 4, 'Pro': 4}} ❌
         
On the one hand, arguments with border security frame can be extracted because they contain more than 8 Pro/Con arguments. On the other hand, arguments with the "economics" frame cannot be extracted or used to create the questionary, as they do not meet the threshold. 

In [49]:
filtered = []
for e in summary:
    a_dict = {}
    extra_dict = {}
    for item, value in e['frame_info'].items():
        if isinstance(value, dict):
            pro = value['stance'].get('Con',0) >= 8 and value['stance'].get('Pro',0) >= 8
            if pro:
                a_dict[item] = value
        else:
            extra_dict[item] = value
    if a_dict:
        a_dict.update(extra_dict)
        b_dict = dict(e)
        b_dict['frame_info'] = a_dict
        filtered.append(b_dict)

In [50]:
print(len(filtered))
filtered

22


[{'topic': '700 mile US Mexico border fence',
  'topic_id': 13,
  'frame_info': {'border security': {'stance': {'Con': 9, 'Pro': 9}},
   'total_pro': 20,
   'total_con': 24,
   'num_of_stances': 44,
   'frames_id': [8, 18, 6, 8, 2, 2]}},
 {'topic': 'Abortion',
  'topic_id': 22,
  'frame_info': {"woman's rights": {'stance': {'Con': 9, 'Pro': 10}},
   'total_pro': 61,
   'total_con': 76,
   'num_of_stances': 137,
   'frames_id': [19,
    20,
    12,
    7,
    7,
    2,
    9,
    10,
    6,
    6,
    6,
    3,
    3,
    2,
    2,
    3,
    2,
    2,
    2,
    2,
    2,
    4,
    1,
    3,
    2]}},
 {'topic': 'Adult male circumcision',
  'topic_id': 27,
  'frame_info': {'adult health': {'stance': {'Con': 8, 'Pro': 11}},
   'total_pro': 34,
   'total_con': 30,
   'num_of_stances': 64,
   'frames_id': [19, 4, 2, 4, 12, 8, 3, 2, 6, 4]}},
 {'topic': 'Animal testing',
  'topic_id': 56,
  'frame_info': {'rights': {'stance': {'Con': 11, 'Pro': 8}},
   'total_pro': 41,
   'total_con': 53,


# What topics can we use for the bias exp? 

with controlling the frame and biased-phrase?


In [51]:
df = pd.read_csv(root+'/data/raw/Webis-argument-framing.csv')
df

,argument_id,frame_id,frame,topic_id,topic,premise,stance,conclusion
0,0,0,economics,0,$700 billion US economic bailout,U.S. Treasury Secretary Henry Paulson summariz...,Pro,Not passing $700b bailout risks sending econom...
1,1,0,economics,0,$700 billion US economic bailout,"""Job security: Safeguarding jobs across the e...",Pro,$700b bailout helps avoid widespread bankruptc...
2,2,0,economics,0,$700 billion US economic bailout,Treasury Secretary Henry Paulson summarized th...,Pro,$700b bailout is generally well designed to so...
3,3,0,economics,0,$700 billion US economic bailout,"""Paulson Plan provides a plan Just as optimis...",Pro,$700b bailout offers buyer for frozen mortgage...
4,4,0,economics,0,$700 billion US economic bailout,"""The Treasury proposal to rescue the financia...",Pro,Most economists support the $700b US economic ...
...,...,...,...,...,...,...,...,...
12321,14110,1622,endangered,1823,Zoos,While some argue that zoos are a means to prot...,Pro,Most of the animals that you see in zoos aren'...
12322,14111,1622,endangered,1823,Zoos,According to the World Conservation Union whic...,Pro,Zoos are not capable of sustaining all endange...
12323,14112,1622,endangered,1823,Zoos,Zoos are a good place to house endangered spec...,Con,Zoos help protect endangered species.
12324,14113,1622,endangered,1823,Zoos,If natural or human factors have made a specie...,Con,Zoos help breed endangered animals.


In [54]:
topics = []
frames = []
final_df = []

for e in filtered:
    frames.append(list(list(e.values())[2].keys())[0])
    topics.append(e['topic_id'])

for x, y in zip(topics, frames):
    temp = df.query('topic_id == "{}" and frame =="{}"'.format(x,y))
    final_df.append(temp)
#final_df

In [55]:
Path(root+'/data/preprocessed/questionnaire_data/').mkdir(parents=True, exist_ok=True)
final_df = pd.concat(final_df)
final_df.to_csv(root+'/data/preprocessed/questionnaire_data/questionnaire_at_least_8_stances.csv', index=False, sep=',')

In [90]:

# def filter_arguments(summary_file, n_stances):
#     filtered = []
#     a_dict = {}
#     extra_dict = {}
#     topics = []
#     frames = []
#     final_df = []
#     df = pd.read_csv(root+'/data/raw/Webis-argument-framing.csv')
#     for e in summary_file:
#         for item, value in e['frame_info'].items():
#             if isinstance(value, dict):
#                 pro = value['stance'].get('Con',0) >= n_stances and value['stance'].get('Pro',0) >= n_stances
#                 if pro:
#                     a_dict[item] = value
#             else:
#                 extra_dict[item] = value
#         if a_dict:
#             a_dict.update(extra_dict)
#             b_dict = dict(e)
#             b_dict['frame_info'] = a_dict
#             filtered.append(b_dict)


#     for e in filtered:
#         frames.append(list(list(e.values())[2].keys())[0])
#         topics.append(e['topic_id'])

#     for x, y in zip(topics, frames):
#         temp = df.query('topic_id == "{}" and frame =="{}"'.format(x,y))
#         final_df.append(temp)
        
#     Path(root+'/data/preprocessed/questionnaire_data/').mkdir(parents=True, exist_ok=True)
#     final_df = pd.concat(final_df)
#     final_df.to_csv(root+'/data/preprocessed/questionnaire_data/{}_questionnaire_at_least_{}_stances.csv'.format(final_df.shape[0],n_stances), index=False, sep=',')
#     return final_df
        


def filter_arguments(summary_file, con_stances, pro_stances):
    filtered = []
    a_dict = {}
    extra_dict = {}
    topics = []
    frames = []
    final_df = []
    df = pd.read_csv(root+'/data/raw/Webis-argument-framing.csv')
    for e in summary_file:
        for item, value in e['frame_info'].items():
            if isinstance(value, dict):
                pro = value['stance'].get('Con',0) >= con_stances and value['stance'].get('Pro',0) >= pro_stances
                if pro:
                    a_dict[item] = value
            else:
                extra_dict[item] = value
        if a_dict:
            a_dict.update(extra_dict)
            b_dict = dict(e)
            b_dict['frame_info'] = a_dict
            filtered.append(b_dict)


    for e in filtered:
        frames.append(list(list(e.values())[2].keys())[0])
        topics.append(e['topic_id'])

    for x, y in zip(topics, frames):
        temp = df.query('topic_id == "{}" and frame =="{}"'.format(x,y))
        final_df.append(temp)
        
    Path(root+'/data/preprocessed/questionnaire_data/').mkdir(parents=True, exist_ok=True)
    final_df = pd.concat(final_df)
    # here fix  
    final_df.to_csv(root+'/data/preprocessed/questionnaire_data/{}_questionnaire_at_least_con_{}_pro_{}_stances.csv'.format(final_df.shape[0],con_stances, pro_stances), index=False, sep=',')
    return final_df



In [86]:
filter_arguments(summary, 4)
filter_arguments(summary, 6)
filter_arguments(summary, 8)
filter_arguments(summary, 10)

,argument_id,frame_id,frame,topic_id,topic,premise,stance,conclusion
3495,3823,62,crime,389,Death penalty,"As a deterrent to others, it depends on how ef...",Pro,Capital punishment has a deterrent effect on c...
3496,3824,62,crime,389,Death penalty,"There are many instances in which released, pa...",Pro,Capital punishment is 100% effective in preven...
3497,3825,62,crime,389,Death penalty,Abolitionists also hold the notion that crimi...,Pro,Criminals fear death and the death penalty
3498,3826,62,crime,389,Death penalty,Life in prison without parole does not protect...,Pro,The death penalty helps protect inmates and pr...
3499,3827,62,crime,389,Death penalty,When the death penalty is a 100% assured punis...,Pro,The death penalty deters crime only if it is a...
...,...,...,...,...,...,...,...,...
9604,10820,62,crime,1182,Prohibition of downloading music and films,"In essence, downloading is using the file with...",Con,Downloading is like taking a photo of a famous...
10181,11508,62,crime,1302,School searches of student lockers,The best way to ensure that such contraband it...,Pro,Locker searches reduce drugs and weapons in sc...
10182,11509,62,crime,1302,School searches of student lockers,"In addition to general searches of lockers, pa...",Pro,Schools must protect against the use of locker...
10183,11510,62,crime,1302,School searches of student lockers,Students should not to be considered criminal ...,Con,"Student locker searches lack ""probable cause""."


In [91]:
filter_arguments(summary, 6, 8)

,argument_id,frame_id,frame,topic_id,topic,premise,stance,conclusion
124,132,28,border security,13,700 mile US Mexico border fence,While the barrier might not cover the entire ...,Pro,The 700 mile fence will help channel illegal i...
125,133,28,border security,13,700 mile US Mexico border fence,Opponents of building the 700-mile wall often ...,Pro,Past fences succeeded in deterring crossings o...
126,134,28,border security,13,700 mile US Mexico border fence,"""...In the mid-1990s, the city was awash in i...",Pro,San Diego is symbol of efficacy of fence
127,135,28,border security,13,700 mile US Mexico border fence,The 700-mile fence would be designed with grea...,Pro,Many past fences have failed to stop illegal i...
128,136,28,border security,13,700 mile US Mexico border fence,"DHS spokesman Jarrod Agen said to , ""It's beyo...",Pro,"Coupling a physical fence with new ""virtual fe..."
129,137,28,border security,13,700 mile US Mexico border fence,There is no substitute for man power. And the ...,Pro,A physical fence accompanied with border patro...
130,138,28,border security,13,700 mile US Mexico border fence,"""Are we not capable of building a fence like ...",Pro,Israeli border fence demonstrates efficacy of ...
131,139,28,border security,13,700 mile US Mexico border fence,Many opponents of a fence point out that it wo...,Pro,A 700-mile fence is only the first step in sec...
132,140,28,border security,13,700 mile US Mexico border fence,Just because the 700-mile fence will be diffi...,Pro,The difficulty of securing the border should n...
133,141,28,border security,13,700 mile US Mexico border fence,"While the wall proposed is 700 miles long, the...",Con,Illegal immigrants will simply go around a 700...
